In [1387]:
import pandas as pd
import numpy as np
import math
import re

## Загрузка датасета

In [2]:
ds = pd.read_csv("hillary-clinton-emails/Emails.csv", 
                 delimiter=",", usecols =["MetadataSubject","ExtractedSubject", "ExtractedBodyText","RawText"])

## Предобработка датасета

В качестве текста исходных писем выбиралось поле ExtractedBodyText. 

In [3816]:
ds.ExtractedBodyText[20]

"H <hrod17@clintonernaii.com›\nWednesday, September 12, 2012 11:26 PM\nesullivanjj@state.gov'\nFw: Fwd: more on libya\nLibya 37 sept 12 12,docx\nWe should get this around asap."

На основе "плохих" частотных слов и структуры исходных писем выделила небольшой список стоп-слов и выражений, которые надо удалить из исходного текста перед обработкой. Например, строки с датой и пр.метаинформация.

In [3817]:
stop_words = ["UNCLASSIFIED", "U.S. Department of State"]
stop_lines = ["case no", "doc no", "importance","cc:", "Cc","Sent from", "h <hrod17@clintonemail.com>", 
              "sunday", "monday","tuesday", "wednesday", "thursday","friday", "saturday"]
stop_phrases = ["@state.gov","@clintonemail","©state.gov"]

Удаляю стоп-слова из писем. Слишком короткие письма не включаю в выборку.

In [3818]:
mails = []
titles = []
for i in range(1, len(ds.index)):
    cur_mail = ""
  
    if type(ds.ExtractedBodyText[i]) is str and len(ds.ExtractedBodyText[i]) > 0:
        lines = ds.ExtractedBodyText[i].split("\n")
        cur_mail = ""
        for line in lines:
            for sl in stop_lines:
                if line.lower().startswith(sl):
                    line = ""
                    break
            for sp in stop_phrases:
                if line.find(sp) > -1:
                    line = ""
                    break
            if re.search('.docx', line) != None:
                line = ""
                
            if len(line) != 0:
                cur_word = line.replace("\n", " ")
                for sw in stop_words:
                    cur_word = cur_word.replace(sw, "")
                cur_mail += " " + cur_word + "."
    if len(cur_mail) > 3:
        mails.append(cur_mail)
    if type(ds.ExtractedSubject[i]) is str and len(ds.ExtractedSubject[i]) > 3:
        titles.append(ds.ExtractedSubject[i].replace("\n", " ") +". ")

### Токенизация

На этом этапе удаляю слишком короткие слова(скорее всего они большой смысловой нагрузки не несут) + использую стеммер

In [3819]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [3820]:
stemmer = PorterStemmer()
stemToWord = {}
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if len(token)>3 and re.match('[A-Za-z]+', token) != None:
            filtered_tokens.append(token.strip("."))
    stems = []
    tags = nltk.pos_tag(filtered_tokens)
    for token, tag in tags:
        stem = stemmer.stem(token)
        if len(stem)>3 and (tag[0] == "N" or tag[0]=="V"):
            stems.append(stem)
            stemToWord[stem]=token
    return stems

In [3821]:
tokenize_and_stem(mails[0])

['latest', 'syria', 'qaddafi', 'march', 'hillari']

In [3822]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

def filterFrequentWords(n, percent, mails):
    vectorizer = CountVectorizer(ngram_range=(n,n),tokenizer=tokenize_and_stem, stop_words=stopwords.words("english"))
    X = vectorizer.fit_transform(mails)
    
    ngramms = defaultdict(int)
    END_OF_SENTENCE = "<END>"
    feature_names = vectorizer.get_feature_names()
    count = [len(np.nonzero(X[0,:])[1]) ]
    wordsCount = 0
    for i in X.nonzero()[1]:
        ngramms[feature_names[i]] += 1
        wordsCount+=1
    
    countToToken = defaultdict(list)
    for key,value in ngramms.items():
        countToToken[value].append(key)
    maxCount = sorted(list(countToToken), reverse=True)
    frequentTokens = [] 
    for c in maxCount:
        frequentTokens += countToToken[c] 
    return frequentTokens[0:int(wordsCount*percent)]
    

## Биграммы

In [3823]:
BiGramms = filterFrequentWords(2, 1, mails)

## Коллокации

In [3824]:
from nltk.collocations import *

In [3825]:
from nltk.tokenize import word_tokenize, sent_tokenize, TweetTokenizer
words = []
END_OF_SENTENCE = "<END>"

for mail in mails:
    words.extend(word_tokenize(mail))
    words.append(END_OF_SENTENCE)

In [3826]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words)
finder.nbest(bigram_measures.pmi, 10) 

[("'1.com", 'aeirig'),
 ("'1HE", ".i't"),
 ("'Banish", 'Sexual'),
 ("'H", 'CCL'),
 ("'Li", 'Matou'),
 ("'Stone", 'Harbour'),
 ("'bad", 'talkers'),
 ("'civ", 'cas'),
 ("'ffrjend", 'nieei'),
 ("'i", 'Voz')]

In [3943]:

def countNGramms(n, mails, indexes):
    model = defaultdict(int)
    
    END_OF_SENTENCE = "<END>"
    for id in indexes:
        words = tokenize_and_stem(mails[id])
            
        for i in range(n-1, len(words)):
            ngramma = "_".join(words[i - (n-1):i+1])
            model[ngramma] += 1
    countToModel = defaultdict(list)
    for key, value in model.items():
        countToModel[value].append(key)
    sortedCount = sorted(countToModel.keys(),reverse=True)
    results = []
    for c in sortedCount[:30]:
        print(c)
        results.extend(countToModel[c])
    return results[:20]
    

## Обучение

Ищу потенциальные стоп-слова - слова, которые встречаются в большом количестве документов(высокая df). Параметром max_df не пользовалась, потому что хотелось посмотреть на частотные слова.

In [3828]:
stopUniGramms = filterFrequentWords(1, 0.0003, mails)


In [3829]:
print(len(stopUniGramms))
print(stopUniGramms)

37
['call', 'work', 'state', 'time', 'want', 'know', 'meet', 'talk', 'secretari', 'today', 'think', 'need', 'make', 'offic', 'tomorrow', 'presid', 'come', 'issu', 'thank', 'take', 'said', 'week', 'year', 'discuss', 'hous', 'peopl', 'help', 'part', 'secur', 'report', 'sent', 'includ', 'govern', 'follow', 'point', 'support', 'hope']


In [3912]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=50000,ngram_range=(1,2),tokenizer=tokenize_and_stem, 
                             stop_words=stopwords.words("english")+stopUniGramms)
X = vectorizer.fit_transform(mails)

In [3913]:

feature_names = vectorizer.get_feature_names()

In [3914]:
len(feature_names)

50000

In [3926]:
n_clusters=8
from sklearn.cluster import KMeans
km = KMeans(n_clusters=n_clusters)


In [3935]:
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## Результаты

Количество писем для каждого кластера. При любых экспериментах выделяется крупный кластер и несколько маленьких. 
Маленькие кластеры характеризуются редкими словами. Большой описывается более частотными.

In [3936]:
km.labels_.sort()
curIndex = 0
curCount = 0
for i in range(len(km.labels_)):
    if km.labels_[i] != curIndex:
        print(curIndex, curCount)
        curCount = 0
        curIndex += 1

    curCount += 1
print(curIndex, curCount)
    

0 129
1 116
2 5458
3 60
4 127
5 103
6 216
7 506


Нашла примеры использования описания центра кластера в качестве характеристики. 

Темы угадываются, но если центр удален, может быть не очень хорошо.

Вообще кажется, что на очень частотные слова смотреть плохо, потому что кластеры могут быть вытянуты по каким-то напрвлениям.

In [3937]:

order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(n_clusters):
    print("Cluster ", i, " centroid")
    print(", ".join([
            " ".join([stemToWord[name] for name in feature_names[ind].split(" ")])
                for ind in order_centroids[i, :10]]))
    print()
    print() 

Cluster  0  centroid
Department, Room, route, Residence, Department Department, conference Room, BENGHAZI, conference, REDACTIONS, REDACTIONS FOIA


Cluster  1  centroid
check, message, assistance, Received, travel, travel check, check message, copies, check assistance, please


Cluster  2  centroid
schedule, morning, confirmed, drafted, list, message, Cheryl, speech, letter, Hillary


Cluster  3  centroid
RELEASE, RELEASE B1,1.4, B1,1.4, RELEASE print, rshah, Gina, mailto, print, Denis, budgets


Cluster  4  centroid
emailed, access emailed, access, travel, please, assistance, emailed assistance, emailed addressing, assistance please, travel access


Cluster  5  centroid
print, print copies, copies, leverage, fine, types, Lauren, influence, Haiti, militant create


Cluster  6  centroid
send, Huma, Abedin, Abedin Huma, Huma Abedin, sheet, lona, copies, Jake, schedule


Cluster  7  centroid
vote, policy, women, plan, Clinton, Obama, parties, countries, election, leaders




In [3944]:

order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(n_clusters):
    print("Cluster ", i, " centroid")
    print(", ".join([
            " ".join([stemToWord[name] for name in feature_names[ind].split(" ")])
                for ind in order_centroids[i, :10]]))
    print()
    print() 

Cluster  0  centroid
DEPART, Room, route, Residence, DEPART DEPART, Conference Room, BENGHAZI, Conference, REDACTIONS, REDACTIONS FOIA


Cluster  1  centroid
checking, message, assistance, Received, travels, travels checking, checking message, copy, checking assistance, please


Cluster  2  centroid
schedule, morning, Confirmed, draft, list, message, Cheryl, speech, letter, Hillary


Cluster  3  centroid
RELEASE, RELEASE B1,1.4, B1,1.4, RELEASE print, rshah, Gina, mailto, print, Denis, budgets


Cluster  4  centroid
emailed, access emailed, access, travels, please, assistance, emailed assistance, emailed address, assistance please, travels access


Cluster  5  centroid
print, print copy, copy, leverage, fine, types, Lauren, Influence, Haiti, militancy create


Cluster  6  centroid
sending, Huma, Abedin, Abedin Huma, Huma Abedin, sheet, Lona, copy, Jake, schedule


Cluster  7  centroid
vote, Policy, women, Planning, Clinton, Obama, party, countries, elections, leaders




Выделяются кластера с темами про выборы; письма для помощников Хиллари Клинтон с какими-то указаниями 

Посмотрим на частотные n-граммы

In [3945]:
curLabel = 0
mailsLabeled = []
curMails = []

indexes = km.labels_.argsort()

for labelInd in indexes:
    label = km.labels_[labelInd]
    if label != curLabel:
        curLabel = label
        mailsLabeled.append(curMails)
        curMails = []
        
   
    curMails.append(labelInd)
mailsLabeled.append(curMails)
for i in range(n_clusters):
    print("Cluster ", i)
    print(", ".join([
            " ".join([stemToWord[v] for v in name.split("_")]) for name in countNGramms(2, mails, mailsLabeled[i])]))
    print()

Cluster  0
82
81
80
17
16
12
11
9
8
7
6
5
4
3
2
1
BENGHAZI COMM, SUBJECT AGREEMENT, PRODUCED HOUSE, DEPT PRODUCED, HOUSE SELECT, SENSITIVE INFORMATION, COMM SUBJECT, FOIA WAIVER, AGREEMENT SENSITIVE, REDACTIONS FOIA, INFORMATION REDACTIONS, SELECT BENGHAZI, STATE DEPT, Date STATE, United STATE, have been, Middle East, president Obama, WAIVER Date, White HOUSE

Cluster  1
127
124
123
56
45
16
15
14
13
12
10
9
8
7
6
5
4
3
2
1
SENSITIVE INFORMATION, INFORMATION REDACTIONS, Subject AGREEMENT, FOIA WAIVER, AGREEMENT SENSITIVE, REDACTIONS FOIA, State DEPT, Benghazi COMM, PRODUCED HOUSE, SELECT Benghazi, COMM Subject, Date State, DEPT PRODUCED, HOUSE SELECT, United State, State Department, Foreign Minister, White HOUSE, Secretary Clinton, Secretary office

Cluster  2
400
376
354
328
309
225
176
163
130
126
121
120
118
116
112
108
101
100
97
96
91
90
87
85
78
77
76
70
69
65
Secretary Office, White House, State DEPART, United State, have been, Secretary State, Private Residence, Health Care, Co

## Оценка

1) Цель: проверить, насколько хорошо связаны слова внутри кластеров

Задание: найдите лишнее слово.

Предлагается 5 слов, описывающих кластер, + слово из другого кластера
Для каждого кластера, получаем величину 1-"Кол-во ошибок/кол-во асессоров"

UPD: Опрос показал, что лучше искать несколько лишних слов.



Проблема с интрепретацией самого большого кластера. 

2) Цель: проверить, насколько хорошо различаются слова из разных кластеров

Задание:  Даны n+1 слов из n  кластеров(при этом 2 слова из одного кластера). Найдите пару.

Для каждого кластера, получаем величину 1- "Кол-во ошибок/кол-во асессоров"

UPD: Лучше выбирать небольшое количество слов. Здесь стало показательным, что кластеризация плохая.


3) Цель: проверить, насколько хорошо связаны слова внутри кластеров

Задание: Дано описание кластера. Исключите лишние слова.


Итог: С 3 заданием сработало лучше всего. В целом кластеризация не очень удалась. 
Очертания некоторых тем выделяются.

Попробуем сделать кластеризацию просто по темам писем